Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.8225398267228907, θ=0.8584304657497792)    …  Distributions.Gamma(α=0.3606401453221473, θ=0.15151542985924848) 
 Distributions.Gamma(α=0.05599631607771682, θ=0.3922461956006653)      Distributions.Gamma(α=0.7087847386107868, θ=0.8291211245866301)  
 Distributions.Gamma(α=0.38429159886572317, θ=0.5725989367646205)      Distributions.Gamma(α=0.017431730376731513, θ=0.7165399480113663)
 Distributions.Gamma(α=0.9587703842060975, θ=0.19297109342034435)      Distributions.Gamma(α=0.7934263128874701, θ=0.19700410740799357) 
 Distributions.Gamma(α=0.19320343883602065, θ=0.00689721672956245)     Distributions.Gamma(α=0.6809037930093542, θ=0.3799426809934008)  )

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.145608  0.682094  0.717286  0.704467 
 0.330691  0.36507   0.760436  0.22653  
 0.229008  0.50359   0.338089  0.0122321)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7733"),[:i,:j],3x4 Array{Float64,2}:
 0.822215  0.599053  0.340469   0.401605
 0.167959  0.445955  0.0301111  0.544941
 0.173532  0.735027  0.806086   0.922456)

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.9013908456106465, σ=0.9057925842112056)    …  Distributions.Normal(μ=0.5191741014593745, σ=0.2948596692203165)   
 Distributions.Normal(μ=0.5975995118397195, σ=0.7659525281619526)       Distributions.Normal(μ=0.806833297995853, σ=0.8745642083959155)    
 Distributions.Normal(μ=0.32274080370260605, σ=0.3563431494088556)      Distributions.Normal(μ=0.020094073128067436, σ=0.10903164334654636)
 Distributions.Normal(μ=0.588565160920703, σ=0.4679676597201161)        Distributions.Normal(μ=0.30569008719145985, σ=0.21757495478936084) 
 Distributions.Normal(μ=0.6424126834124049, σ=0.007147020953811145)     Distributions.Normal(μ=0.16605745163452656, σ=0.8623412191342432)  )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.0552068  0.524623  0.156848  0.737092
 0.958227   0.675943  0.240383  0.471283
 0.770456   0.373231  0.709254  0.522452)

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-35.36597143116796

In [10]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-173.3965810005201,-13.593072740907873,0.0449816833123928)

In [14]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,[:i,:j],20x6 Array{Float64,2}:
 0.904512   0.0884127  0.167459    0.686012   0.181552  0.459253
 0.022691   0.778186   0.541389    0.626844   0.397803  0.411249
 0.295733   0.704995   0.0518976   0.6275

In [15]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [16]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [17]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…

In [18]:
vars = fieldnames(obs)
macroexpand(:(@wrapvars $vars $(naturals(typeof(obs), Val{:x}, Normal)) (i_1, i_2)))

quote  # /Users/jmxp/code/vbgraph/types.jl, line 335:
    (Eμ,Eτ) = (E(project(f,:μ,(i_1,i_2))),E(project(f,:τ,(i_1,i_2)))) # /Users/jmxp/code/vbgraph/types.jl, line 336:
    (Eμ .* Eτ,-Eτ / 2)
end

In [19]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (12.617885859516003,-11.000000000000002)
 (9.437346427583906,-11.000000000000002) 
 (9.64163321182307,-11.000000000000002)  
 (11.876559994678296,-11.000000000000002)
 (11.427606303774803,-11.000000000000002)
 (12.601778355646031,-11.000000000000002)

In [19]:
naturals(obs, Val{:τ}, τ)

LoadError: LoadError: MethodError: `naturals` has no method matching naturals(::VB.LogNormalFactor{2}, ::Type{Val{:τ}}, ::VB.RandomNode{Distributions.Gamma})
while loading In[19], in expression starting on line 1

In [20]:
method_exists(naturals, (LogNormalFactor, Type{Val{:x}}, RandomNode{Normal}))

false

In [21]:
check_conjugate(m.nodes[:τ])

LoadError: LoadError: UndefVarError: check_conjugate not defined
while loading In[21], in expression starting on line 1

In [22]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [23]:
update!(μ, Val{:conjugate})

LoadError: LoadError: UndefVarError: update! not defined
while loading In[23], in expression starting on line 1

In [24]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)